In [1]:
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator

In [2]:
# loading data
raw = pd.read_csv("forestfires.csv")
data = pd.DataFrame(raw)
data.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [3]:
# prepocessing data
#make dummy variables from X,Y,Month,Day
data_with_dummies = pd.get_dummies(data, columns=["X","Y"], prefix=["X","Y"])
data_with_dummies = pd.get_dummies(data_with_dummies, columns=["month","day"])
data_with_dummies.head()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,X_1,...,month_nov,month_oct,month_sep,day_fri,day_mon,day_sat,day_sun,day_thu,day_tue,day_wed
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,0,...,0,0,0,1,0,0,0,0,0,0
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,...,0,1,0,0,0,0,0,0,1,0
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0,...,0,1,0,0,0,1,0,0,0,0
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,0,...,0,0,0,1,0,0,0,0,0,0
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0,...,0,0,0,0,0,0,1,0,0,0


In [4]:
# create graph for the model
#TODO: Jesse check of dit model hetzelfde is als wat we hadden
G = BayesianModel()
G.add_nodes_from(data_with_dummies)
G.add_edges_from([("temp","area"),
                  ("wind","area"),
                  ("rain","area"),
                  ("RH","area"),
                  ("FFMC","area"),
                  ("DMC","area"),
                  ("ISI","area"),
                  ("RH","FFMC"),
                  ("wind","FFMC"),
                  ("rain","FFMC"),
                  ("temp","FFMC"),
                  ("RH","DMC"),
                  ("rain","DMC"),
                  ("temp","DMC"),
                  ("FFMC","ISI"),
                  ("wind","ISI")])
G.add_edges_from([(col,"area") for col in data_with_dummies if col.startswith('X')])
G.add_edges_from([(col,"rain") for col in data_with_dummies if col.startswith('Y')])
G.add_edges_from([(col,"RH") for col in data_with_dummies if col.startswith('day')])
G.add_edges_from([(col,"temp") for col in data_with_dummies if col.startswith('month')])
G.add_edges_from([(col,"wind") for col in data_with_dummies if col.startswith('month')])

In [ ]:
# fit model on data
#Duurt lang, in 15 min nog niet klaar (of error msg) - laptop wordt wel erg warm.
G.fit(data_with_dummies) # only used for discrete bayesnets?